## İBB Açık Verileri Görselleştirme
* İbb'nin yayınlamış olduğu veri setlerinden coğrafi verileri harita yapmak için kullandım.
* Bu veriler;kart dolum merkezleri,ispark otoparkları ve sağlık kuruluşları.
* Amacımız verileri harita üzerinde nokta olarak göstermek,aralarındaki mesafeyi öğrenmek ve yeni nokta eklemek.

### Veri kayanğı için:
* https://data.ibb.gov.tr
* https://www.kaggle.com/mielek/health-institutions-datas-of-istanbul
* https://www.kaggle.com/abdulkerimnee/ibb-ak-veri
* https://www.kaggle.com/abdulkerimnee/ibb-kart-dolum-merkezleri


* Kütüphanelerimizi ekleyelim.

In [ ]:
import folium 
import geopandas as gpd
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import CustomIcon
from folium.plugins import MeasureControl
from folium.plugins import Draw

* Veri setlerimizi okuyalım.

In [ ]:
veri_ispark=pd.read_excel('../input/ibb-ak-veri/ispark-otoparklarna-ait-bilgiler.xlsx')
veri_kart=pd.read_excel('../input/ibb-kart-dolum-merkezleri/istanbulkart-dolum-merkezi-bilgileri.xlsx')
veri_sağlık=pd.read_csv('../input/health-institutions-datas-of-istanbul/salk-kurum-ve-kurulularna-ait-bilgiler.csv',sep=',',encoding='latin-1')

* Veri setlerinde bulunan enlem ve boylam sütunlarında Nan değerlerini ilerde hata almamak için kaldıralım.

In [ ]:
veri_ispark=veri_ispark.dropna(subset=['Enlem'])
veri_ispark=veri_ispark.dropna(subset=['Boylam'])
veri_kart=veri_kart.dropna(subset=["LONGITUDE"])
veri_kart=veri_kart.dropna(subset=["LATITUDE"])
veri_sağlık=veri_sağlık.dropna(subset=['ENLEM'])
veri_sağlık=veri_sağlık.dropna(subset=['BOYLAM'])

* Kart dolum merkezleri veri setinde İstanbul iliniin dışında olan noktaları kaldıralım.
* Bu noktaları daha önceden harita üzerinde görüntülüyerek kendim belirledim.

In [ ]:
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==28.663911].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==24.10052475].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==29.844776].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==28.9853446].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==29.27964].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==41.036632].index, inplace=True)
veri_kart.drop(veri_kart.loc[veri_kart.LONGITUDE==28.100232].index, inplace=True)

* Veri setlerinde bulunan enlem ve boylam sütunlarını liste formatına çevirelim.


In [ ]:
ispark=veri_ispark[['Enlem','Boylam']].values.tolist()
kart=veri_kart[['LATITUDE','LONGITUDE']].values.tolist()
sağlık=veri_sağlık[['ENLEM','BOYLAM']].values.tolist()

* `folium` ile temel altlık haritamızı oluşturalım.

In [ ]:
m=folium.Map(location=(41.1944, 28.9651),
            zoomstart=9,tiles='openstreetmap')
m

* Altlık haritalara farklı katmanlar ekleyelim.

In [ ]:
folium.TileLayer('cartodbdark_matter').add_to(m);
folium.TileLayer('cartodbpositron').add_to(m);

* Uydu görüntüsü ekleme.

In [ ]:
folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m);

* Veri setlerimizi tabaka olarak ekleyelim.

In [ ]:
tabaka1=folium.FeatureGroup(name="İspark Otopark")
tabaka2=folium.FeatureGroup(name='Kart Dolum Merkezleri')
tabaka3=folium.FeatureGroup(name='Sağlık Kuruluşları')

* Veri setlerimizdeki noktalar fazla olduğu için `MarkerCluster` parametresi ile  herbirini kendi içinde gruplayalım.

In [ ]:
marker_cluster =MarkerCluster(data=ispark).add_to(tabaka1)
for point in range(len(ispark)):
    İSPARK= folium.Marker(ispark[point],popup=str('Park ID'),
                          icon=folium.Icon(color='blue', icon='car',prefix='fa')).add_to(marker_cluster)

In [ ]:
marker_cluster=MarkerCluster(data=kart).add_to(tabaka2)
for point in range(len(kart)):
    KART=folium.Marker(kart[point],
                       icon=folium.Icon(color='green', icon='ticket',prefix='fa')).add_to(marker_cluster)

In [ ]:
marker_cluster=MarkerCluster(data=sağlık).add_to(tabaka3)
for point in range(len(sağlık)):
    SAĞLIK=folium.Marker(sağlık[point],
                        icon=folium.Icon(color='red',icon='heartbeat',prefix='fa')).add_to(marker_cluster)

* En son olarak tüm katmanlarımızı temel haritamıza ekleyelim.
* Lüften harita çıktısında sağ taraftan istediğiniz altlık haritayı seçiniz.

In [ ]:
draw = Draw()#çizim araçlarını oluşturma
draw.add_to(m)#çizim araçlarını haritaya ekleme
m.add_child(tabaka1)#tabakaları haritaya ekleme
m.add_child(tabaka2)
m.add_child(tabaka3)
m.add_child(MeasureControl())#mesafeleri ölçmek için
m.add_child(folium.map.LayerControl())
m.save('index.html')
m